<h2 align="left"><font color='red'> 8-Puzzle Problem </font></a></h2>  

![](https://ece.uwaterloo.ca/~dwharder/aads/Algorithms/N_puzzles/images/puz3.png)

&emsp; &emsp; **<font color='blue'> Start state</front>**        &emsp;&emsp; &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;      **<font color='green'> Goal state</front>**

<font color='black'> </front>
<b>This was a task given by the school where we had to complete the following tasks:
</b>

The 8-puzzle is a sliding puzzle that consists of a frame of numbered square tiles from 1 to 8 in random order with one tile missing. The objective of the puzzle is to place the tiles in order by making sliding moves that use the empty space.

A sliding tile puzzle where you can swap the blank with an adjacent piece, trying to reach a goal configuration. The cells are numbered 0 to 8, starting at the top left and going row by row left to right. The pieces are numebred 1 to 8, with 0 representing the blank. 

Note that this puzzle also exists in other sizes, for example 3-puzzle (2x2), 15-puzzle (4x4), 24-puzzle (5x5), etc.

 1. <font color='darkred'>Formulate the 8-puzzle problem (implement a class EightPuzzle)</front>
 2. <font color='darkred'>Implement uniform cost search for this example</front>
 3. <font color='darkred'>Implement two heuristics for the problem (misplaced tiles and manhattan distance)</front>
 4. <font color='darkred'>Implement a greedy search that reaches the goal state</front>
 5. <font color='darkred'>Implement A* search considering two different heuristics</front>

1. <font color='darkred'>Formulate the 8-puzzle problem (implement a class EightPuzzle)</front>

In [22]:
import random
from search import *
from Complementary_notebook import *
from FunUtil import *

In [23]:
class EightPuzzle:
    """
        We are creating a class EightPuzzle to formulate the 8-puzzle problem.
    """

    """ 
        In our case, a state is represented by a 3x3 matrix with values ranging from 1-8 placed inside the matrix and an 
        empty space represented by a 0. The initial state is a matrix 3x3 where the values and empty space are randomly 
        placed inside the matrix. The goal state is when the values are correctly placed. 
        Initial State:
        [
            [5, 2, 7],
            [8, 4, 0],
            [1, 3, 6]
        ]

        Goal State (this is an example of a goal state):
        [
            [1, 2, 3],
            [4, 5, 6],
            [7, 8, 0]
        ]
    """
    def __init__(self, initial, goal = None):
        self.initial = initial
        self.goal = goal

    """
        An action is when the empty tile (value) slides from one spot (index) to another. There can be multiple ways of moving 
        the empty tile (value) based on its position in the matrix. You can slide to the left, right, up and down. This method
        will return the set of actions that can be executed in the given state.
    """
    def actions(self, state):
        actions = []
        for i in range(3):
            for j in range(3):
                if state[i][j] == 0:
                    if i == 0 and j == 0:
                        actions.append('right')
                        actions.append('down')
                    elif i == 0 and j == 1:
                        actions.append('left')
                        actions.append('right')
                        actions.append('down')
                    elif i == 0 and j == 2:
                        actions.append('left')
                        actions.append('down')
                    elif i == 1 and j == 0:
                        actions.append('up')
                        actions.append('right')
                        actions.append('down')
                    elif i == 1 and j == 1:
                        actions.append('up')
                        actions.append('left')
                        actions.append('right')
                        actions.append('down')
                    elif i == 1 and j == 2:
                        actions.append('up')
                        actions.append('left')
                        actions.append('down')
                    elif i == 2 and j == 0:
                        actions.append('up')
                        actions.append('right')
                    elif i == 2 and j == 1:
                        actions.append('up')
                        actions.append('left')
                        actions.append('right')
                    elif i == 2 and j == 2:
                        actions.append('up')
                        actions.append('left')

        return actions
    
    """
        This method will return the state that results from executing the given action in the given state.
    """
    def result(self, state, action):
        new_state = [row[:] for row in state]
        for i in range(3):
            for j in range(3):
                if state[i][j] == 0:
                    if action == 'up':
                        new_state[i][j] = new_state[i - 1][j]
                        new_state[i - 1][j] = 0
                    elif action == 'down':
                        new_state[i][j] = new_state[i + 1][j]
                        new_state[i + 1][j] = 0
                    elif action == 'left':
                        new_state[i][j] = new_state[i][j - 1]
                        new_state[i][j - 1] = 0
                    elif action == 'right':
                        new_state[i][j] = new_state[i][j + 1]
                        new_state[i][j + 1] = 0
        return new_state

    """
        This method will return True if the state is a goal state, and False otherwise.
    """
    def goal_test(self, state):
        return state == self.goal

    """
        This method will return the cost of a solution path that arrives at state2 from state1 via action, assuming cost 1
        for each step in the path.
    """
    def path_cost(self, c, state1, action, state2):
        return c + 1

    """
        This method will return a random state.
    """
    def random_state(self):
        state = [row[:] for row in self.initial]
        actions = []
        for i in range(100):
            actions = self.actions(state)
            action = random.choice(actions)
            state = self.result(state, action)
        return state

    def display(self, state):
        for i in range(3):
            for j in range(3):
                print(state[i][j], end = ' ')
            print()
        print()

In [24]:
array = [
    [1,2,3],
    [4,5,6],
    [7,8,0] 
]


problem = EightPuzzle(array)
problem.initial = problem.random_state()
problem.goal = [
    [1,2,3],
    [4,0,6],
    [7,5,8] 
]

problem.display(problem.initial)
problem.display(problem.goal)

7 1 2 
8 0 3 
5 6 4 

1 2 3 
4 0 6 
7 5 8 



 2. <font color='darkred'>Implement uniform cost search for this example</front>

In [25]:
"""
    We created these 2 methods because the uniform cost search algorithm the explored set only accepts hashable objects.
"""

# Create a function that converts the list to a string
def convertToString(list):
    string = ""
    for i in range(3):
        for j in range(3):
            string += str(list[i][j])
    return string

In [26]:
def best_first_graph_search(problem, f, display=False):
    """Search the nodes with the lowest f scores first.
    You specify the function f(node) that you want to minimize; for example,
    if f is a heuristic estimate to the goal, then we have greedy best
    first search; if f is node.depth then we have breadth-first search.
    There is a subtlety: the line "f = memoize(f, 'f')" means that the f
    values will be cached on the nodes as they are computed. So after doing
    a best first search you can examine the f values of the path returned."""
    f = memoize(f, 'f')
    node = Node(problem.initial)
    frontier = PriorityQueue('min', f)
    frontier.append(node)
    explored = set()
    while frontier:
        node = frontier.pop()
        if problem.goal_test(node.state):
            if display:
                print(len(explored), "paths have been expanded and", len(frontier), "paths remain in the frontier")
            return node
        explored.add(convertToString(node.state))
        for child in node.expand(problem):
            if convertToString(child.state) not in explored and child not in frontier:
                frontier.append(child)
            elif child in frontier:
                if f(child) < frontier[child]:
                    del frontier[child]
                    frontier.append(child)
    return None


def uniform_cost_search(problem, display=False):
    return best_first_graph_search(problem, lambda node: node.path_cost, display)

In [27]:
# Implement uniform cost search from the search.py file
ucs = uniform_cost_search(problem)

In [28]:
# We can observe that the algorithm reached the goal state
ucs

<Node [[1, 2, 3], [4, 0, 6], [7, 5, 8]]>

 3. <font color='darkred'>Implement two heuristics for the problem (misplaced tiles and manhattan distance)</front>

In [29]:
# Implement a function to calculate the misplaced tiles heuristic
# Implement a function to calculate the misplaced tiles heuristic
def misplaced_tiles(node):
    goal_state = problem.goal # Goal state configuration
    misplaced = 0
    for i in range(len(node.state)):
        for j in range(len(node.state[i])):
            if node.state[i][j] != goal_state[i][j]:
                misplaced += 1
    return misplaced

In [30]:
# Implement a function to calculate the manhattan distance heuristic
def manhattan_distance(node):
    goal_state = problem.goal  # Goal state configuration
    distance = 0
    for i in range(len(node.state)):
        for j in range(len(node.state[i])):
            if node.state[i][j] != goal_state[i][j] and node.state[i][j] != 0:
                num = node.state[i][j]
                goal_row, goal_col = divmod(num - 1, 3)
                distance += abs(i - goal_row) + abs(j - goal_col)
    return distance

 4. <font color='darkred'>Implement a greedy search that reaches the goal state</front>

In [31]:
#greedy_best_first_graph_search = best_first_graph_search

def greedy_best_first_graph_search(problem, h=None, display=False):
    """ best-first graph search with f(n) = h(n).
    You need to specify the h function when you call best-first graph search, or
    else in your Problem subclass."""
    h = memoize(h or problem.h, 'h')
    return best_first_graph_search(problem, lambda n: h(n), display)

In [32]:
# Greedy search that reaches the goal state
greedy = best_first_graph_search(problem, misplaced_tiles)

In [33]:
greedy

<Node [[1, 2, 3], [4, 0, 6], [7, 5, 8]]>

 5. <font color='darkred'>Implement A* search considering two different heuristics</front>

In [34]:
def astar_search(problem, h=None, display=False):
    """A* search is best-first graph search with f(n) = g(n)+h(n).
    You need to specify the h function when you call astar_search, or
    else in your Problem subclass."""
    h = memoize(h or problem.h, 'h')
    return best_first_graph_search(problem, lambda n: n.path_cost + h(n), display)


In [35]:
# Hueristic 1: Misplaced tiles
print("Misplaced tiles heuristic")
a_star_misplaced_tiles = astar_search(problem, misplaced_tiles)
a_star_misplaced_tiles

Misplaced tiles heuristic


<Node [[1, 2, 3], [4, 0, 6], [7, 5, 8]]>

In [36]:
# Heuristic 2: Manhattan distance
print("Manhattan distance heuristic")
a_star_manhattan_distance = astar_search(problem, manhattan_distance)
a_star_manhattan_distance

Manhattan distance heuristic


<Node [[1, 2, 3], [4, 0, 6], [7, 5, 8]]>